In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # import tqdm for progress bar 
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [2]:
target_labels = ['Peach___Bacterial_spot', 'Peach___healthy']

In [3]:
main_path = 'data_penyakit/dataset/train'

In [4]:
# Load the images and labels
filename = []
X = []
y = []
for label in target_labels:
    label_dir = os.path.join(main_path, label)
    for img_file in os.listdir(label_dir):
        img_path = os.path.join(label_dir, img_file)
        img = load_img(img_path, target_size=(128, 128))
        img_array = img_to_array(img)
        filename.append(img_file)
        X.append(img_array)
        y.append(target_labels.index(label))

In [5]:
from tensorflow.keras.utils import to_categorical

In [6]:
X = np.array(X)
y = to_categorical(y, num_classes=len(target_labels))

In [7]:
X.shape

(3566, 128, 128, 3)

In [8]:
y.shape

(3566, 2)

In [9]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test, filename_train, filename_test = train_test_split(X, y, filename, test_size=0.2, random_state=42)

In [10]:
X_train.shape

(2852, 128, 128, 3)

In [11]:
X_test.shape

(714, 128, 128, 3)

In [12]:
y_train.shape

(2852, 2)

In [13]:
y_test.shape

(714, 2)

In [14]:
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.models import Sequential

In [15]:
len(target_labels)

2

In [16]:
cnn_model = Sequential()
cnn_model.add(Conv2D(32, kernel_size= (3,3), activation = 'relu',input_shape=(128,128,3)))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(96,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(32,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())

cnn_model.add(Dropout(0.2))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation = 'relu'))
cnn_model.add(Dropout(0.3))
cnn_model.add(Dense(len(target_labels), activation='softmax'))
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
# Train the CNN model
cnn_history = cnn_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

Epoch 1/10


2023-05-31 14:08:18.396980: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


90/90 [==============================] - 46s 492ms/step - loss: 0.1486 - accuracy: 0.9435 - val_loss: 1.2619 - val_accuracy: 0.7493
Epoch 2/10
90/90 [==============================] - 56s 626ms/step - loss: 0.0620 - accuracy: 0.9772 - val_loss: 0.3920 - val_accuracy: 0.9076
Epoch 3/10
90/90 [==============================] - 57s 639ms/step - loss: 0.0602 - accuracy: 0.9800 - val_loss: 0.0308 - val_accuracy: 0.9916
Epoch 4/10
90/90 [==============================] - 41s 458ms/step - loss: 0.0462 - accuracy: 0.9825 - val_loss: 0.0504 - val_accuracy: 0.9888
Epoch 5/10
90/90 [==============================] - 39s 438ms/step - loss: 0.0295 - accuracy: 0.9895 - val_loss: 0.0679 - val_accuracy: 0.9818
Epoch 6/10
90/90 [==============================] - 41s 456ms/step - loss: 0.0308 - accuracy: 0.9888 - val_loss: 0.0464 - val_accuracy: 0.9860
Epoch 7/10
90/90 [==============================] - 40s 440ms/step - loss: 0.0349 - accuracy: 0.9874 - val_loss: 0.0557 - val_accuracy: 0.9860
Epoch 8/10

In [18]:
# Evaluate the performance of the trained CNN model on the test set
cnn_scores = cnn_model.evaluate(X_test, y_test, verbose=0)
print("CNN Model Accuracy: %.2f%%" % (cnn_scores[1] * 100))

CNN Model Accuracy: 96.78%


In [19]:
cnn_scores

[0.07755421847105026, 0.9677870869636536]

In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Use the model to predict the test dataset
y_pred_prob = cnn_model.predict(X_test)

# Convert probabilities to classes
y_pred = np.argmax(y_pred_prob, axis=1)

# If y_test is one-hot encoded, convert it to classes as well
y_test_classes = np.argmax(y_test, axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_test_classes, y_pred)

# Calculate precision
precision = precision_score(y_test_classes, y_pred, average='weighted') 

# Calculate recall
recall = recall_score(y_test_classes, y_pred, average='weighted') 

# Calculate F1 score
f1 = f1_score(y_test_classes, y_pred, average='weighted')

# Print the metrics
print("Accuracy: %.2f%%" % (accuracy * 100))
print("Precision: %.2f%%" % (precision * 100))
print("Recall: %.2f%%" % (recall * 100))
print("F1 score: %.2f%%" % (f1 * 100))

23/23 [==============================] - 3s 105ms/step
Accuracy: 96.78%
Precision: 96.93%
Recall: 96.78%
F1 score: 96.77%


In [21]:
from tabulate import tabulate

# Create a DataFrame with true labels, predicted labels, and target labels
df = pd.DataFrame({'Filename': filename_test,
                   'True Labels': [target_labels[label] for label in y_test_classes],
                   'Predicted Labels': [target_labels[label] for label in y_pred]})

# Convert DataFrame to tabular format
table = tabulate(df, headers='keys', tablefmt='psql')

# Print the table
print(table)

# Save the DataFrame to a CSV file
df.to_csv('labels_predictions-cnn-peach.csv', index=False)

+-----+-------------------------------------------------------------------------+------------------------+------------------------+
|     | Filename                                                                | True Labels            | Predicted Labels       |
|-----+-------------------------------------------------------------------------+------------------------+------------------------|
|   0 | b26d10da-345c-4a42-b7f1-7e4eca71b37e___Rutg._Bact.S 1714.JPG            | Peach___Bacterial_spot | Peach___Bacterial_spot |
|   1 | 74fb7a01-8e8d-4b5e-99a0-22b9a2bdf837___Rutg._HL 3591_newPixel25.JPG     | Peach___healthy        | Peach___healthy        |
|   2 | 0ec60f21-e376-46df-9ae3-517ca6494eba___Rut._Bact.S 1213.JPG             | Peach___Bacterial_spot | Peach___Bacterial_spot |
|   3 | 3212700f-4294-4c19-a2de-a984920bad2e___Rut._Bact.S 0829.JPG             | Peach___Bacterial_spot | Peach___Bacterial_spot |
|   4 | 3ebeec7d-28e5-4203-a92f-20adb3fc2a45___Rutg._Bact.S 2345.JPG        

In [22]:
# Filter rows where true label and predicted label do not match
false_predictions = df[df['True Labels'] != df['Predicted Labels']]

# Convert DataFrame to tabular format
table = tabulate(false_predictions, headers='keys', tablefmt='psql')

# Print the false predictions table
print("False Predictions:")
print(table)

# Save the false predictions to a CSV file
false_predictions.to_csv('false_predictions-cnn-peach.csv', index=False)

False Predictions:
+-----+-----------------------------------------------------------------+------------------------+------------------------+
|     | Filename                                                        | True Labels            | Predicted Labels       |
|-----+-----------------------------------------------------------------+------------------------+------------------------|
|  37 | d9822f7b-564d-4d8a-8aa5-e8b30eb4884b___Rutg._HL 3681_180deg.JPG | Peach___healthy        | Peach___Bacterial_spot |
|  51 | bec8c39f-e3c9-4ea2-9e89-85e6f9654219___Rutg._HL 3643_180deg.JPG | Peach___healthy        | Peach___Bacterial_spot |
|  87 | d3fdde62-a86b-4ac9-bf87-c898df257417___Rutg._HL 3592_270deg.JPG | Peach___healthy        | Peach___Bacterial_spot |
|  89 | 7ce55677-2a3e-4dc1-b16b-f006d53c19ac___Rutg._HL 3603.JPG        | Peach___healthy        | Peach___Bacterial_spot |
| 114 | 1b00ca3e-6b4f-4ab8-a566-7acb3039ee44___Rut._Bact.S 3422.JPG     | Peach___Bacterial_spot | Peach___health

In [23]:
# Extract the features using the trained CNN model
cnn_features = cnn_model.predict(X_train)

90/90 [==============================] - 10s 110ms/step


# rf

In [24]:
import time

In [25]:
# import the Random Forest classifier
from RF_Manual import RandomForest

In [26]:
# Train a Random Forest classifier on the extracted features
start_time = time.time()
rf_classifier = RandomForest(n_trees=100, max_depth=42)
rf_classifier.fit(cnn_features, np.argmax(y_train, axis=1))

# count time for training
end_time = time.time()
training_time = end_time - start_time
print(f"Training time: {training_time} seconds")

Training time: 11.745912790298462 seconds


In [27]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Evaluate the performance of the Random Forest classifier on the test set
rf_features = cnn_model.predict(X_test)
rf_features = rf_features.reshape(rf_features.shape[0], -1)
rf_predictions = rf_classifier.predict(rf_features)
rf_accuracy = np.mean(rf_predictions == np.argmax(y_test, axis=1))
print("Random Forest Classifier Accuracy: %.2f%%" % (rf_accuracy * 100))

# Calculate and print the Precision
rf_precision = precision_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Precision: %.2f" % rf_precision)

# Calculate and print the Recall
rf_recall = recall_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Recall: %.2f" % rf_recall)

# Calculate and print the F1 score
rf_f1_score = f1_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier F1 Score: %.2f" % rf_f1_score)

23/23 [==============================] - 3s 133ms/step
Random Forest Classifier Accuracy: 98.88%
Random Forest Classifier Precision: 0.99
Random Forest Classifier Recall: 0.99
Random Forest Classifier F1 Score: 0.99


In [28]:
# Create a DataFrame with true labels and predicted labels
df_rf = pd.DataFrame({'Filename': filename_test,
                      'True Labels': [target_labels[label] for label in y_test_classes],
                      'Predicted Labels': [target_labels[label] for label in rf_predictions]})

# Convert DataFrame to tabular format
table_rf = tabulate(df_rf, headers='keys', tablefmt='psql')

# Print the table
print(table_rf)

# Save the DataFrame to a CSV file
df_rf.to_csv('labels_predictions-rf-peach.csv', index=False)

+-----+-------------------------------------------------------------------------+------------------------+------------------------+
|     | Filename                                                                | True Labels            | Predicted Labels       |
|-----+-------------------------------------------------------------------------+------------------------+------------------------|
|   0 | b26d10da-345c-4a42-b7f1-7e4eca71b37e___Rutg._Bact.S 1714.JPG            | Peach___Bacterial_spot | Peach___Bacterial_spot |
|   1 | 74fb7a01-8e8d-4b5e-99a0-22b9a2bdf837___Rutg._HL 3591_newPixel25.JPG     | Peach___healthy        | Peach___healthy        |
|   2 | 0ec60f21-e376-46df-9ae3-517ca6494eba___Rut._Bact.S 1213.JPG             | Peach___Bacterial_spot | Peach___Bacterial_spot |
|   3 | 3212700f-4294-4c19-a2de-a984920bad2e___Rut._Bact.S 0829.JPG             | Peach___Bacterial_spot | Peach___Bacterial_spot |
|   4 | 3ebeec7d-28e5-4203-a92f-20adb3fc2a45___Rutg._Bact.S 2345.JPG        

In [29]:
# Filter rows where true label and predicted label do not match
false_predictions_rf = df_rf[df_rf['True Labels'] != df_rf['Predicted Labels']]

# Convert DataFrame to tabular format
table_rf = tabulate(false_predictions_rf, headers='keys', tablefmt='psql')

# Print the false predictions table
print("False Predictions:")
print(table_rf)

# Save the false predictions to a CSV file
false_predictions_rf.to_csv('false_predictions-rf-peach.csv', index=False)

False Predictions:
+-----+-----------------------------------------------------------------+------------------------+------------------------+
|     | Filename                                                        | True Labels            | Predicted Labels       |
|-----+-----------------------------------------------------------------+------------------------+------------------------|
|  21 | c4cf1b6a-7d05-4064-86ec-6224afb6d936___Rut._Bact.S 0914.JPG     | Peach___Bacterial_spot | Peach___healthy        |
|  37 | d9822f7b-564d-4d8a-8aa5-e8b30eb4884b___Rutg._HL 3681_180deg.JPG | Peach___healthy        | Peach___Bacterial_spot |
| 106 | a667eaee-845d-49b5-8350-2789f4f0cc73___Rut._Bact.S 0990.JPG     | Peach___Bacterial_spot | Peach___healthy        |
| 114 | 1b00ca3e-6b4f-4ab8-a566-7acb3039ee44___Rut._Bact.S 3422.JPG     | Peach___Bacterial_spot | Peach___healthy        |
| 170 | a83faf6a-2aa7-4bc3-8709-f5acc6255bd7___Rutg._HL 3678.JPG        | Peach___healthy        | Peach___Bacter